In [31]:
import requests
import json
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [32]:
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/96bebbfc76104c5e923d42fb149b60d7'))
api_key = '76GSRJKVXQSTN6PZ3R1UM54PDXV4QYHPZP'

In [33]:
lower_case_address = '0x269616d549d7e8eaa82dfb17028d0b212d11232a'
contract_address = Web3.toChecksumAddress(lower_case_address)
with open('2nd_abi.json','r') as f:
    conract_abi = json.load(f)


In [34]:
contract = w3.eth.contract(address=contract_address, abi=conract_abi['result'])

# Retrieve the total supply of NFTs
allValidNFTs = contract.functions.totalSupply().call()

print("Total NFTs in the vault:", allValidNFTs//10**18)

Total NFTs in the vault: 137


In [50]:
api_endpoint = 'https://api.etherscan.io/api'
api_key = '76GSRJKVXQSTN6PZ3R1UM54PDXV4QYHPZP'
module = 'account'
action = 'txlist'
address = contract_address
start_block = 0
end_block = 'latest'
sort = 'asc'

In [56]:
url = f"{api_endpoint}?module={module}&action={action}&address={address}&startblock={start_block}&endblock={end_block}&sort={sort}&apikey={api_key}"
response = requests.get(url).json()
# Extract the total supply from each transaction
total_supplies = []
for tx in response['result']:
    tx_hash = tx['hash']
    block_number = tx['blockNumber']
    block = w3.eth.getBlock(block_number)
    timestamp = block.timestamp
    topics = tx['topics']
    for topic in topics:
        if topic.startswith('0x'):
            try:
                total_supply = int(topic, 16)
                total_supplies.append((timestamp, total_supply))
                break
            except ValueError:
                pass

# Plot the historical number of NFTs in the contract
timestamps, supplies = zip(*total_supplies)
plt.plot(timestamps, supplies)
plt.xlabel('Timestamp')
plt.ylabel('Total NFTs')
plt.show()

ValueError: {'code': -32602, 'message': 'invalid argument 0: hex string without 0x prefix'}